In [1]:
import fourierseries
import util
import phaser
import dataloader
# Preprocess data for a single subject - to be send to modeling frameworks
def find_phase(k):
    """
    Detrend and compute the phase estimate using Phaser
    INPUT:
      k -- dataframe
    OUTPUT:
      k -- dataframe
    """
    #l = ['hip_flexion_l','hip_flexion_r'] # Phase variables = hip flexion angles
    y = np.array(k)
    print(y.shape)
    y = util.detrend(y.T).T
    print(y.shape)
    phsr = phaser.Phaser(y=y)
    k[:] = phsr.phaserEval(y)[0,:]
    return k


Running phaser


In [2]:
import os
import scipy.io
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchdiffeq import odeint_adjoint as odeint
from torch.utils.data import DataLoader
os.environ['KMP_DUPLICATE_LIB_OK']='True'


samp_trajs_TE = torch.load('samp_trajs_TE_nonoise_Xcoord_tau6k9.pt')
samp_trajs_val_TE = torch.load('samp_trajs_val_TE_nonoise_Xcoord_tau6k9.pt')

tau = 6
k = 9
mesured_dim = 10

trial_num = 401

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#batch = 1500 #for lstm512
batch = 3000 #for lstm256

ts_num = 0.25
tot_num = 50

samp_ts = np.linspace(0, ts_num, num=tot_num)
samp_ts = torch.from_numpy(samp_ts).float().to(device)

orig_trajs_TE = np.load('orig_trajs_TE_tau6k9_Xcoord.npy')
orig_trajs_TE = orig_trajs_TE.reshape(trial_num, 200*34, mesured_dim*(k+1)+2)
samp_trajs_TE_test = orig_trajs_TE[:, :tot_num, :]
samp_trajs_TE_test = torch.from_numpy(samp_trajs_TE_test).float().to(device).reshape(trial_num, tot_num, mesured_dim*(k+1)+2)

#Load to Dataloader
train_loader = DataLoader(dataset = samp_trajs_TE, batch_size = batch, shuffle = True, drop_last = True)
val_loader = DataLoader(dataset = samp_trajs_TE, batch_size = batch, shuffle = True, drop_last = True)


class LatentODEfunc(nn.Module):

    def __init__(self, latent_dim=8, nhidden=50):
        super(LatentODEfunc, self).__init__()
        #self.tanh = nn.ELU(inplace= True)
        self.tanh = nn.Tanh()
        self.fc1 = nn.Linear(latent_dim, nhidden)
        self.fc2 = nn.Linear(nhidden, nhidden)
        self.fc3 = nn.Linear(nhidden, latent_dim)
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        return out

class RecognitionRNN(nn.Module):

    def __init__(self, latent_dim=8, obs_dim=46, nhidden=50, nbatch=1):
        super(RecognitionRNN, self).__init__()
        self.nhidden = nhidden
        self.nbatch = nbatch
        #self.h1o = nn.Linear(obs_dim, 8)
        self.h1o = nn.Linear(obs_dim, 20)
        self.h3o = nn.Linear(20, latent_dim*2)
        self.lstm = nn.LSTMCell(latent_dim*2, nhidden)
        self.tanh = nn.Tanh()
        self.h2o = nn.Linear(nhidden, latent_dim*2)

    def forward(self, x, h, c):
        xo = self.h1o(x)
        xo = self.tanh(xo)
        xxo = self.h3o(xo)
        hn, cn = self.lstm(xxo, (h,c))
        hn = self.tanh(hn)
        out = self.h2o(hn)
        return out, hn, cn
    

    def initHidden(self):
        return torch.zeros(1, self.nbatch, self.nhidden)


class Decoder(nn.Module):

    def __init__(self, latent_dim=8, obs_dim=46, nhidden=50):
        super(Decoder, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.tanh = nn.Tanh()
        self.fc1 = nn.Linear(latent_dim, nhidden)
        self.fc2 = nn.Linear(nhidden, nhidden*2)
        self.fc3 = nn.Linear(nhidden*2, obs_dim)

    def forward(self, z):
        out = self.fc1(z)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        return out


class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val


def log_normal_pdf(x, mean, logvar):
    const = torch.from_numpy(np.array([2. * np.pi])).float().to(x.device)
    const = torch.log(const)
    return -.5 * (const + logvar + (x - mean) ** 2. / torch.exp(logvar))

def mseloss(x, mean):
    loss = nn.MSELoss()
    return loss(x, mean)

def normal_kl(mu1, lv1, mu2, lv2):
    v1 = torch.exp(lv1)
    v2 = torch.exp(lv2)
    lstd1 = lv1 / 2.
    lstd2 = lv2 / 2.

    kl = lstd2 - lstd1 + ((v1 + (mu1 - mu2) ** 2.) / (2. * v2)) - .5
    return kl

def MSELoss(yhat, y):
    assert type(yhat) == torch.Tensor
    assert type(y) == torch.Tensor
    return torch.mean((yhat - y) ** 2)


def get_args():
    return {'latent_dim': latent_dim,
            'obs_dim': obs_dim,
            'nhidden': nhidden,
            'dec_nhidden' : dec_nhidden,
            'rnn_nhidden': rnn_nhidden,
            'device': device}

def get_state_dicts():
    return {'odefunc_state_dict': func.state_dict(),
            'encoder_state_dict': rec.state_dict(),
            'decoder_state_dict': dec.state_dict()}

def data_get_dict():
    return {
        'samp_trajs_TE': samp_trajs_TE,
        'samp_trajs_val_TE': samp_trajs_val_TE,
        'samp_ts': samp_ts,
    }

def get_losses():
    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_losses_k1': val_losses_k1,
        'val_losses_k2': val_losses_k2,
        'val_losses_k3': val_losses_k3,
        'val_losses_k4': val_losses_k4,
        'val_losses_k5': val_losses_k5,
        'val_losses_k6': val_losses_k6,
        'val_losses_k7': val_losses_k7,
        'val_losses_k8': val_losses_k8,
        'val_losses_k9': val_losses_k9,
    }

def save_model(tau, k, latent_dim, itr):

    save_dict = {
        'model_args': get_args(),
        'optimizer_state_dict': optimizer.state_dict(),
        #'data': data_get_dict(),
        'train_loss': get_losses()
    }
    
    save_dict.update(get_state_dicts())
    
    torch.save(save_dict, 'model/ODE_TakenEmbedding_rnn2_lstm{}_tau{}k{}_LSTM_lr0.008_latent{}_LSTMautoencoder_Dataloader_epoch{}.pth'.format(rnn_nhidden, tau, k, latent_dim, itr))

    
def data_for_plot_graph(gen_index):
    with torch.no_grad():
        # sample from trajectorys' approx. posterior

        ts_pos = np.linspace(0, 0.25*gen_index, num=50*gen_index)
        ts_pos = torch.from_numpy(ts_pos).float().to(device)
    
        h = torch.zeros(1, samp_trajs_TE_test.shape[0], rnn_nhidden).to(device)
        c = torch.zeros(1, samp_trajs_TE_test.shape[0], rnn_nhidden).to(device)
    
        hn = h[0, :, :]
        cn = c[0, :, :]
    
        for t in reversed(range(samp_trajs_TE_test.size(1))):
            obs = samp_trajs_TE_test[:, t, :]
            out, hn, cn = rec.forward(obs, hn, cn)
        qz0_mean, qz0_logvar = out[:, :latent_dim], out[:, latent_dim:]
        epsilon = torch.randn(qz0_mean.size()).to(device)
        z0 = epsilon * torch.exp(.5 * qz0_logvar) + qz0_mean

        # forward in time and solve ode for reconstructions
        pred_z = odeint(func, z0, ts_pos).permute(1, 0, 2) #change time and batch with permute
        pred_x = dec(pred_z)
        
        return pred_x, pred_z
    
def plot_graph(gen_index, times_index, dataset_value, deriv_index, pred_x_forgraph, orig_trajs, itr, path):
    with torch.no_grad():
        orig_trajs_forgraph = orig_trajs
        ts_pos_combined = np.linspace(0, 0.25*gen_index, num=50*gen_index) 
        
        fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15, 9)) #####MAKE SURE ROW COL MATCHES THE NUM OF FEATURES
        axes = axes.flatten()
        
        for i, ax in enumerate(axes):
            ax.scatter(ts_pos_combined[times_index:times_index+50*gen_index], orig_trajs_forgraph[dataset_value,times_index:50*gen_index, i*(k+1)+deriv_index], label='sampled data', s = 5)
            ax.plot(ts_pos_combined[times_index:times_index+50*gen_index], pred_x_forgraph[dataset_value, times_index:times_index+50*gen_index, i*(k+1)+deriv_index], 'r',
                 label='learned trajectory (t>0)')
            ax.set_ylim(-2.5, 2.5)

        plt.legend()
        plot_name = 'lstm_datasetnum{}_latent{}_gen{}_deriv{}_epoch{}.png'.format(dataset_value, latent_dim, gen_index, deriv_index, itr)
        save_path = os.path.join(path, plot_name)
        plt.savefig(save_path, dpi=500)
        plt.close()
    



        
latent_dim = 4
nhidden = 64
dec_nhidden = 20
obs_dim = 10*(k+1)+2
rnn_nhidden = 256
nitrs = 3000
noise_std = 0.2

func = LatentODEfunc(latent_dim, nhidden).to(device)
rec = RecognitionRNN(latent_dim, obs_dim, rnn_nhidden, batch).to(device)
dec = Decoder(latent_dim, obs_dim, dec_nhidden).to(device)
params = (list(func.parameters()) + list(dec.parameters()) + list(rec.parameters()))
optimizer = optim.Adam(params, lr=0.008)
loss_meter = RunningAverageMeter()


checkpoint = torch.load('model/ODE_Xcoord_TakenEmbedding_rnn2_lstm256_tau6k9_LSTM_lr0.008_latent4_LSTMautoencoder_Dataloader_epoch315.pth')
rec.load_state_dict(checkpoint['encoder_state_dict'])
func.load_state_dict(checkpoint['odefunc_state_dict'])
dec.load_state_dict(checkpoint['decoder_state_dict'])

<All keys matched successfully>

In [3]:
A = np.load('RodentGait2500predict_noise1_total_trial20.npy')

In [4]:
A.shape

(401, 50000, 10)

In [5]:
B = np.load('subs.npy')

In [7]:
B.shape

(401, 1)

In [5]:
gen_index = 50
itr=315
deriv_index = 5
times_index = 0
tot_index = 30

orig_trajs = orig_trajs_TE[:, 0:0+tot_num*gen_index, :]

pred_x, pred_z = data_for_plot_graph(gen_index)
pred_x = pred_x.reshape(trial_num, tot_num*gen_index, mesured_dim*(k+1)+2)
pred_x_forgraph = pred_x.detach().cpu().numpy()

pred_x_unormalize = pred_x_forgraph[:, : , deriv_index::mesured_dim][:,:,:mesured_dim]
pred_z_forgraph = pred_z.detach().cpu().numpy()

In [11]:
def load_data_un_normalize(pred_x_forgraph, mesured_dim, datafilepath):
    #datafilepath = 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/RodentXcoord_fps200.npy'
    data = np.load(datafilepath)
    traj_tot = np.load(datafilepath).reshape(trial_num, 9900, mesured_dim)
    traj_tot = traj_tot[:,2000:9000,:]
    data = data[:, 2000:9000, :]
    pred_x_unormalize = pred_x_forgraph[:, : , deriv_index::k][:,:,:mesured_dim]
    

    X = np.zeros((data.shape[0],pred_x_unormalize.shape[1],data.shape[2]))
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            trajs = pred_x_unormalize[i,:,j]
            trajs_tot = traj_tot[i,:,j]
            X[i,:,j] = (trajs * trajs_tot.std())+ trajs_tot.mean()
            
    #samp_trajs += npr.randn(*samp_trajs.shape) * noise_std #add noise

    return X

In [8]:
X = load_data_un_normalize(pred_x_forgraph, mesured_dim, 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/RodentXcoord_fps200.npy')

## Add Noise

In [4]:
def data_for_plot_graph_noise(gen_index, noise_interval):
    with torch.no_grad():

        ts_pos = np.linspace(0, ts_num*gen_index, num=tot_num*gen_index)
        ts_pos = torch.from_numpy(ts_pos).float().to(device)
    
        h = torch.zeros(1, samp_trajs_TE_test.shape[0], rnn_nhidden).to(device)
        c = torch.zeros(1, samp_trajs_TE_test.shape[0], rnn_nhidden).to(device)
    
        hn = h[0, :, :]
        cn = c[0, :, :]
    
        for t in reversed(range(samp_trajs_TE_test.size(1))):
            obs = samp_trajs_TE_test[:, t, :]
            out, hn, cn = rec.forward(obs, hn, cn)
        qz0_mean, qz0_logvar = out[:, :latent_dim], out[:, latent_dim:]
        epsilon = torch.randn(qz0_mean.size()).to(device)
        z0 = epsilon * torch.exp(.5 * qz0_logvar) + qz0_mean

        # forward in time and solve ode for reconstructions
        pred_z = odeint(func, z0, ts_pos).permute(1, 0, 2) #change time and batch with permute
        for i in range(pred_z.shape[1]):   
            if i%noise_interval == 0:
                pred_z[:,i,:] = pred_z[:,i,:] + torch.rand(pred_z.shape[0],pred_z.shape[2]).to(device)*noise_std #add noise
        pred_x = dec(pred_z)
        
        return pred_x, pred_z

In [14]:
noise_std = 1
gen_index = 50
itr=315
deriv_index = 5
times_index = 0
tot_index = 50
noise_interval = 5
orig_trajs = orig_trajs_TE[:, 0:0+50*gen_index, :]

pred_x_noise, pred_z_noise = data_for_plot_graph_noise(tot_index, noise_interval)
pred_x_noise = pred_x_noise.reshape(trial_num, tot_num*tot_index, mesured_dim*(k+1)+2)
pred_z_noise = pred_z_noise.reshape(trial_num, tot_num*tot_index, latent_dim)
pred_x_noise_forgraph = pred_x_noise.detach().cpu().numpy()
pred_z_noise_forgraph = pred_z_noise.detach().cpu().numpy()

In [20]:
pred_x_forgraph_deriv_5_noise = load_data_un_normalize(pred_x_noise_forgraph, mesured_dim, 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/RodentXcoord_fps200.npy')
np.save("RodentGait2500predict_noise{}.npy".format(noise_std), pred_x_forgraph_deriv_5_noise)

## Plot

In [21]:
def plot_graph_noise_nonoise(gen_index, times_index, tot_index, dataset_value, deriv_index, pred_x_forgraph, orig_trajs, itr, path):
    with torch.no_grad():
        orig_trajs_forgraph = orig_trajs
        ts_pos_combined = np.linspace(0, ts_num*tot_index, num=tot_num*tot_index) 
        
        fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15, 9))
        axes = axes.flatten()
        
        for i, ax in enumerate(axes):
            ax.plot(ts_pos_combined[times_index:times_index+tot_num*tot_index], pred_x_forgraph[dataset_value,times_index:tot_num*tot_index, i*(k+1)+deriv_index], 'r')
            ax.plot(ts_pos_combined[times_index:times_index+tot_num*tot_index], pred_x_noise_forgraph[dataset_value, times_index:times_index+tot_num*tot_index, i*(k+1)+deriv_index], 'b')
            ax.set_ylim(-3, 3)

        
        plot_name = 'Noise_vs_Nonoise_datasetnum{}_latent{}_gen{}_deriv{}_epoch{}.png'.format(dataset_value, latent_dim, tot_index, deriv_index, itr)
        save_path = os.path.join(path, plot_name)
        plt.savefig(save_path, dpi=500)
        plt.close()

In [23]:
path = "NoiseComparison/tau{}k{}/longtimeseries/latent{}/Trial2_RLONG_data_loader_rnn2layer_lstm{}_lr0.008_timestep{}/epoch{}".format(tau, k, latent_dim, rnn_nhidden, tot_num, itr)

if not os.path.exists(path):
   os.makedirs(path)

for i in range(trial_num):
    plot_graph_noise_nonoise(gen_index, times_index, tot_index, i, deriv_index, pred_x_forgraph, orig_trajs, itr, path)

KeyboardInterrupt: 

In [24]:
pred_x_forgraph_deriv_4_noise.shape

(203, 2500, 31)

In [7]:
measured_dim

NameError: name 'measured_dim' is not defined

## Experiment for Multiple Trials

In [12]:
Multiple_Trial_Index = 20

for i in range(Multiple_Trial_Index):
    noise_std = 1
    gen_index = 50
    itr=315
    deriv_index = 5
    times_index = 0
    tot_index = 50
    noise_interval = 5
    orig_trajs = orig_trajs_TE[:, 0:0+50*gen_index, :]

    pred_x_noise, pred_z_noise = data_for_plot_graph_noise(tot_index, noise_interval)
    pred_x_noise = pred_x_noise.reshape(trial_num, tot_num*tot_index, mesured_dim*(k+1)+2)
    pred_z_noise = pred_z_noise.reshape(trial_num, tot_num*tot_index, latent_dim)
    pred_x_noise_forgraph = pred_x_noise.detach().cpu().numpy()
    pred_z_noise_forgraph = pred_z_noise.detach().cpu().numpy()
    
    pred_x_forgraph_deriv_5_noise = load_data_un_normalize(pred_x_noise_forgraph, mesured_dim, 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/RodentXcoord_fps200.npy')
    np.save("RodentGait{}predict_noise{}_trial{}.npy".format(tot_num*tot_index,noise_std, i), pred_x_forgraph_deriv_5_noise)
    

In [13]:
Multiple_Trials_Concatenated = []
for i in range(Multiple_Trial_Index):
    Upload_Data = np.load("RodentGait2500predict_noise{}_trial{}.npy".format(noise_std, i))
    if i == 0:
        Multiple_Trials_Concatenated = Upload_Data.copy()
    else:
        Multiple_Trials_Concatenated = np.append(Multiple_Trials_Concatenated, Upload_Data, axis = 1)
        
np.save("RodentGait{}predict_noise{}_total_trial{}.npy".format(tot_num*tot_index, noise_std, Multiple_Trial_Index), Multiple_Trials_Concatenated)